In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names

https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s

In [31]:
dataset=pd.read_csv('amazon_reviews_utf.csv')

In [5]:
dataset

,Name,Star Rating,Verifed Review,Helpful Vote Count,Date,Author,Author Link,Review Link,Review Text
0,"So far, so good (and dry)",5,YES,25,"November 9, 2016",S. R. Southard,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3V8GI...,I had my first opportunity to use the umbrella...
1,Not really a lifetime warranty...,2,YES,On,"July 10, 2018",Rico,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1ZA3B...,I bought this umbrella a few months ago and un...
2,"""THIS IS THE ONLY UMBRELLA YOU SHOULD EVER PUR...",5,YES,On,"January 22, 2018",On The Fly,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/RG5V0S...,First of all: I have no financial interest in ...
3,Most good quality umbrellas can't stand up to ...,3,YES,2,"April 25, 2018",Linann,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R2EYVB...,"I work in New Haven CT, and in the fall, and w..."
4,Decent umbrella compromised by poor-quality ri...,3,YES,3,"October 15, 2017",Edward Ripley-Duggan,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1L9TU...,The umbrella appears to be robust; I used it r...
5,it is still like new. Great construction,4,YES,On,"August 14, 2016",MC Oddslice,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R2W29X...,"Update: Too small! If there is wind, you will ..."
6,For a tiny person,2,YES,On,"May 25, 2018",AR,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3FPDT...,I was excited to try out this umbrella based o...
7,"Decent, a bit pricey?",4,YES,2,"September 2, 2017",Alex Wang,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R12Q4A...,This umbrella is pretty good. never had a umbr...
8,Not consistent with reviews or description.,2,YES,3,"June 20, 2017",Hannah,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1UX1K...,I must have received a bad item or something b...
9,Seems well-built and well-designed.,5,YES,0,"January 26, 2018",Terende,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3C9IS...,I bought two of these for my and my spouse's u...


In [17]:
X= dataset.drop(['Star Rating'], axis=1)

In [19]:
y=dataset['Star Rating']

<h1>Emotion Analysis</h1>

<h2>Options</h2>

It may not be necessary to get emotions from an API. I could do the non-ML way of determining emotion, by tallying up the total emotions from every word in an input then dividing by len(words) to get average emotions. Take only the float values of those 7? emotions at the end and assign that to a new column in reviews df. Then the X becomes the review itself and the y becomes the combination of the star rating and the emotion scores. Then make a self organizing map out of that. 

depeche=pd.read_csv('DepecheMood_V1.0/DepecheMood_normfreq.txt', sep='\t')
depeche_word_isolation=depeche.copy()
depeche_word_isolation['Lemma#PoS']=depeche_word_isolation['Lemma#PoS'].str.split('#').str.get(0)
depeche_word_isolation.to_csv('depeche_word_isolation.csv', index=False)

In [35]:
emotions=pd.read_csv('depeche_word_isolation.csv')

Need to get the column/series name for the row column.

Method one: idxmax. Method two: lambda apply.

In [ ]:
emotions.iloc[0:10].apply(lambda x: heapq.nlargest(2, x[1:10]), axis=1)

<h2>Threshold of 0.3</h2>

In [95]:
emotions['max_emotion']=emotions.max(axis=1)
emotions=emotions[emotions['max_emotion']>0.3]

<h2>Threshold of 0.5</h2>

In [4]:
emotions['max_emotion']=emotions.max(axis=1)
emotions=emotions[emotions['max_emotion']>0.5]

In [5]:
emotions_no_words=emotions.drop(['Lemma#PoS'], axis=1)
emotions['Dominant_emotion']=emotions_no_words.idxmax(axis=1)

<h3>Remove numbers</h3>

In [ ]:
import re

remove=re.compile(r'[0-9][10-999]')
for index, row in emotions.iloc[0:10].iterrows():
    print(row)
    if re.search(remove, row['Lemma#PoS']):
        emotions=emotions.drop(index, axis=0)
    

The problem is all entries are strings so we can't extract the numbers

In [16]:
afraid=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AFRAID'])
amused=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AMUSED'])
angry=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANGRY'])
annoyed=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANNOYED'])
dont_care=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='DONT_CARE'])
happy=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='HAPPY'])
inspired=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='INSPIRED'])
sad=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='SAD'])

def word_feats(word):
    return dict([(letter, True) for letter in word])

afraid_features=[(word_feats(str(word)), 'afraid') for word in afraid]
amused_features=[(word_feats(str(word)), 'amused') for word in amused]
angry_features=[(word_feats(str(word)), 'angry') for word in angry]
annoyed_features=[(word_feats(str(word)), 'annoyed') for word in annoyed]
dont_care_features=[(word_feats(str(word)), 'dont_care') for word in dont_care]
happy_features=[(word_feats(str(word)), 'happy') for word in happy]
inspired_features=[(word_feats(str(word)), 'inspired') for word in inspired]
sad_features=[(word_feats(str(word)), 'sad') for word in sad]

train_set=afraid_features+amused_features+angry_features+annoyed_features+dont_care_features+happy_features+inspired_features+sad_features
classifier=NaiveBayesClassifier.train(train_set)

In [57]:
afraid=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AFRAID'])
amused=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AMUSED'])
angry=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANGRY'])
annoyed=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANNOYED'])
dont_care=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='DONT_CARE'])
happy=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='HAPPY'])
inspired=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='INSPIRED'])
sad=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='SAD'])

def word_feats(word):
    output = []
    for character in word:
        number = float(ord(character) - 96)
        output.append(number)
    return output

In [59]:
word_feats('one_hundred_and_eleven')

[15.0,
 14.0,
 5.0,
 -1.0,
 8.0,
 21.0,
 14.0,
 4.0,
 18.0,
 5.0,
 4.0,
 -1.0,
 1.0,
 14.0,
 4.0,
 -1.0,
 5.0,
 12.0,
 5.0,
 22.0,
 5.0,
 14.0]

In [61]:
X_list=[]
y_list=[]
X=np.matrix([])

emotions_dict={0: 'afraid', 1: 'amused', 2: 'angry', 3: 'annoyed', 4: 'dont_care', 5: 'happy', 6: 'inspired', 7: 'sad'}
for index, emotion in enumerate([afraid, amused, angry, annoyed, dont_care, happy, inspired, sad]):
    for word in emotion:
        letters=word_feats(str(word))
        letters_arr=np.array(letters)
        print(letters_arr)
        X+=letters
        y_list.append(emotions_dict[index])

y=np.matrix(y_list)

[-47. -48. -48.  20.   8.]


ValueError: operands could not be broadcast together with shapes (1,0) (5,) (1,0) 

I'm gonna try my original, non-numpy idea now

In [3]:
from imblearn.over_sampling import SMOTE 

sm=SMOTE(random_state=1, ratio='minority', k_neighbors=5, m_neighbors=10, 
         out_step=0.5, kind='regular', n_jobs=-1)

In [44]:
X_res, y_res= sm.fit_sample(X, y)

ValueError: setting an array element with a sequence.

I need to create X and y, the data that have to be sampled and corresponding label for each sample in X. 

I'll do undersampling first because it's easier.

In [64]:
import random

afraid=random.sample(afraid, 3200)
amused=random.sample(amused, 3200)
angry=random.sample(angry, 3200)
dont_care=random.sample(dont_care, 3200)
happy=random.sample(happy, 3200)
inspired=random.sample(inspired, 3200)
sad=random.sample(sad, 3200)

In [126]:
import random

afraid=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AFRAID'])
amused=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AMUSED'])
angry=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANGRY'])
annoyed=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANNOYED'])
dont_care=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='DONT_CARE'])
happy=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='HAPPY'])
inspired=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='INSPIRED'])
sad=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='SAD'])

emotions_list=[afraid, amused, angry, annoyed, dont_care, happy, inspired, sad]
emotions_lengths=[]
for emotion in emotions_list:
    emotions_lengths.append(len(emotion))
smallest_length=min(emotions_lengths)

afraid=random.sample(afraid, smallest_length)
amused=random.sample(amused, smallest_length)
angry=random.sample(angry, smallest_length)
dont_care=random.sample(dont_care, smallest_length)
happy=random.sample(happy, smallest_length)
inspired=random.sample(inspired, smallest_length)
sad=random.sample(sad, smallest_length)

def word_feats(word):
    return dict([(letter, True) for letter in word])

afraid_features=[(word_feats(str(word)), 'afraid') for word in afraid]
amused_features=[(word_feats(str(word)), 'amused') for word in amused]
angry_features=[(word_feats(str(word)), 'angry') for word in angry]
annoyed_features=[(word_feats(str(word)), 'annoyed') for word in annoyed]
dont_care_features=[(word_feats(str(word)), 'dont_care') for word in dont_care]
happy_features=[(word_feats(str(word)), 'happy') for word in happy]
inspired_features=[(word_feats(str(word)), 'inspired') for word in inspired]
sad_features=[(word_feats(str(word)), 'sad') for word in sad]

train_set=afraid_features+amused_features+angry_features+annoyed_features+dont_care_features+happy_features+inspired_features+sad_features

In [ ]:
classifier=NaiveBayesClassifier.train(train_set)

In [140]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [ ]:
classifier=SklearnClassifier(BernoulliNB()).train(train_set)

In [141]:
classifier=SklearnClassifier(SVC(), sparse=False).train(train_set)

In [148]:
classifier.classify(word_feats('test'))

'dont_care'

In [147]:
predict('This umbrella sucks! I hate it!')

Afraid: 0.16666666666666666
Amused: 0.16666666666666666
Angry: 0.0
Annoyed: 0.0
Don't care: 0.16666666666666666
Happy: 0.0
Inspired: 0.3333333333333333
Sad: 0.16666666666666666


Now the code seems to be giving me different results based on what I put in but it doesn't seem very accurate.

I need to probably train a neural network. 

In [65]:
len(afraid)

3200

In [66]:
len(amused)

3200

In [67]:
len(angry)

3200

In [330]:
len(annoyed)

3200

In [331]:
len(dont_care)

4200

In [332]:
len(happy)

4201

In [333]:
len(inspired)

7266

In [334]:
len(sad)

4159

<h1>The source of the error</h1>

More of one label of training data skews the classifier that direction. How to solve? Give equal inputs of each emotion. (Oversampling in this case)

First I'll try with my own method of oversampling... then SMOTE. 

In [321]:
classifier.classify(word_feats('sad'))

'amused'

In [281]:
id(afraid_features)

314959688

In [282]:
id(inspired_features)

328492744

<h1>'float' object is not iterable</h1>

In [202]:
classifier.show_most_informative_features()

Most Informative Features
                       1 = True            happy : inspir =     12.9 : 1.0
                       2 = True            happy : angry  =     10.3 : 1.0
                       0 = True              sad : inspir =      8.8 : 1.0
                       9 = True            happy : inspir =      8.2 : 1.0
                       4 = True            happy : inspir =      7.1 : 1.0
                       8 = True            happy : inspir =      5.6 : 1.0
                       7 = True            happy : amused =      5.3 : 1.0
                       5 = True            happy : sad    =      5.1 : 1.0
                       . = True            angry : sad    =      4.0 : 1.0
                       3 = True            happy : afraid =      3.1 : 1.0


In [197]:
classifier.classify(word_feats('adfjasld;fkj'))

'inspired'

In [198]:
len(inspired)

6968

In [199]:
len(afraid)

4977

In [200]:
len(amused)

6147

In [201]:
len(dont_care)

4276

DEFUNCT

In [112]:
import heapq
for index, row in emotions.iloc[0:10].iterrows():
    values=[]
    for value in row.iloc[1:10]:
        values.append(value)
    largest_values=heapq.nlargest(2, values)
    print(largest_values)

[0.291063671, 0.167116461]
[0.164304729, 0.13883705]
[0.143611016, 0.138484847]
[0.170863303, 0.170658442]
[0.257339157, 0.15571317199999998]
[0.586477761, 0.17265862199999998]
[0.4134563120000001, 0.336853157]
[0.320738407, 0.146574328]
[1.0, 0.0]
[0.209132296, 0.18620568]


In [110]:
def predict(sentence):
    afraid=0
    amused=0
    angry=0
    annoyed=0
    dont_care=0
    happy=0
    inspired=0
    sad=0
    sentence = sentence.lower()
    words = sentence.split(' ')
    for word in words:
        classResult = classifier.classify(word_feats(word))
        if classResult == 'afraid':
            afraid+=1
        if classResult == 'amused':
            amused+=1
        if classResult == 'angry':
            angry+=1
        if classResult == 'annoyed':
            annoyed+=1
        if classResult == 'dont_care':
            dont_care+=1
        if classResult == 'happy':
            happy+=1
        if classResult == 'inspired':
            inspired+=1
        if classResult == 'sad':
            sad+=1
    print('Afraid: ' + str(float(afraid)/len(words)))
    print('Amused: ' + str(float(amused)/len(words)))
    print('Angry: '  + str(float(angry)/len(words)))
    print('Annoyed: '+ str(float(annoyed)/len(words)))
    print('Don\'t care: ' + str(float(dont_care)/len(words)))
    print('Happy: '  + str(float(happy)/len(words)))
    print('Inspired: '+ str(float(inspired)/len(words)))
    print('Sad: '+ str(float(sad)/len(words)))

In [87]:
new=predict('I am very angry')
new

Afraid: 0.0
Amused: 0.0
Angry: 0.0
Annoyed: 0.5
Don't care: 0.0
Happy: 0.0
Inspired: 0.5
Sad: 0.0


Now I'm getting bogus results. I need to (maybe) redo taking the dominant emotions to only include those with high values.

In [253]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
 
def word_feats(word):
    return dict([(letter, True) for letter in word])
 
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(' ]
neutral_vocab = [ 'movie','the','sound','was','is','actors','did','know','words','not' ]
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
 
train_set = negative_features + positive_features + neutral_features
classifier = NaiveBayesClassifier.train(train_set) 
 
# Predict
neg = 0
pos = 0
sentence = "bbbbb"
sentence = sentence.lower()
words = sentence.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1
 
print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))

Positive: 0.0
Negative: 1.0


<h1>Ideas to fix bug</h1>

1. Take only words with relatively high emotional components
2. Get features differently (different method of tokenizing? words)
3. 

<h1>Easy peasy non-ML way of emotions</h1>

In [8]:
afraid_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AFRAID'])
amused_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='AMUSED'])
angry_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANGRY'])
annoyed_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='ANNOYED'])
dont_care_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='DONT_CARE'])
happy_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='HAPPY'])
inspired_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='INSPIRED'])
sad_li=list(emotions['Lemma#PoS'][emotions['Dominant_emotion']=='SAD'])

In [57]:
def predict(sentence):
    afraid=0.0
    amused=0
    angry=0
    annoyed=0
    dont_care=0
    happy=0
    inspired=0
    sad=0
    sentence = sentence.lower()
    words = sentence.split(' ')
    for word in words:
        if word in list(emotions['Lemma#PoS']):
            afraid+=float(emotions['AFRAID'][emotions['Lemma#PoS']==word])
        if word in amused_li:
            amused+=1
        if word in angry_li:
            angry+=1
        if word in annoyed_li:
            annoyed+=1
        if word in dont_care_li:
            dont_care+=1
        if word in happy_li:
            happy+=1
        if word in inspired_li:
            inspired+=1
        if word in sad_li:
            sad+=1
    print('Afraid: ' + str((afraid)/len(words)))
    print('Amused: ' + str(float(amused)/len(words)))
    print('Angry: '  + str(float(angry)/len(words)))
    print('Annoyed: '+ str(float(annoyed)/len(words)))
    print('Don\'t care: ' + str(float(dont_care)/len(words)))
    print('Happy: '  + str(float(happy)/len(words)))
    print('Inspired: '+ str(float(inspired)/len(words)))
    print('Sad: '+ str(float(sad)/len(words)))

In [58]:
predict('I am afraid')

Afraid: 0.04542982333333334
Amused: 0.0
Angry: 0.0
Annoyed: 0.0
Don't care: 0.0
Happy: 0.0
Inspired: 0.0
Sad: 0.0


In [59]:
emotions[emotions['Lemma#PoS']=='afraid']

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
737,afraid,0.0842,0.159583,0.135333,0.132405,0.074147,0.09452,0.179293,0.140519


In [ ]:
depeche_tf_idf=pd.read_csv('DepecheMood_V1.0/DepecheMood_normfreq.txt', sep='\t')

<h1>Taking the Training over Labelled Dataset Approach</h1>

In [75]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2, axis=1)

In [76]:
isear

,0,1
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...
5,shame,When I realized that I was directing the feeli...
6,guilt,I feel guilty when when I realize that I consi...
7,joy,After my girlfriend had taken her exam we went...
8,fear,"When, for the first time I realized the meanin..."
9,anger,When a car is overtaking another and I am forc...


In [86]:
isear[0][isear[0]=='guit']='guilt'
isear[0].value_counts()

joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: 0, dtype: int64

In [88]:
'''
A class that performs emotional classification
The class uses the ISEAR dataset as the training model for predicting the emotional tag for a given text
It also returns the probabilistic value for any given text
'''
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [97]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [98]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [102]:
text=[]
for index, row in isear.iterrows():
    text.append(clean(row[1]))

In [103]:
text

['day feel close partner friend feel peace also experience close contact people regard greatly',
 'every time imagine someone love could contact serious illness even death',
 'obviously unjustly treated possibility elucidating this',
 'think short time live relate period life think use short time',
 'gathering found involuntarily sitting next two people expressed opinion considered low discriminating',
 'realized directing feeling discontent partner way trying put blame instead sorting feeliings',
 'feel guilty realize consider material thing important caring relative feel selfcentered',
 'girlfriend taken exam went parent place',
 'when first time realized meaning death',
 'car overtaking another forced drive road',
 'recently thought hard work take study one want try something else read theoretical book english understand',
 'found bristle liver paste tube',
 'tired unmotivated shouted girlfriend brought negative side character actually important',
 'think study enough weekend think 

In [108]:
y=np.array(isear[0])

array(['joy', 'fear', 'anger', ..., 'fear', 'guilt', 'fear'], dtype=object)

In [109]:
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [110]:
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
#vector = vectorizer.transform(input_text)
# summarize encoded vector
X=vector.todense()

In [111]:
encoder=LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)
y=np_utils.to_categorical(y)
y

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]], dtype=float32)

In [126]:
encoder.inverse_transform([0, 4, 0, 0, 0, 0, 0])

array(['anger', 'joy', 'anger', 'anger', 'anger', 'anger', 'anger'], dtype=object)

In [131]:
y[0:10]

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [132]:
isear.iloc[0:10]

,0,1
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...
5,shame,When I realized that I was directing the feeli...
6,guilt,I feel guilty when when I realize that I consi...
7,joy,After my girlfriend had taken her exam we went...
8,fear,"When, for the first time I realized the meanin..."
9,anger,When a car is overtaking another and I am forc...


0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [115]:
#without wrapper
classifier=Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer (Softmax because we want probabilities within 0 and 1 )
classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN (gradient descent with logarithmic loss)
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [116]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
6012/6012 [==============================] - 1s 141us/step - loss: 1.9451 - acc: 0.1467
Epoch 2/100
6012/6012 [==============================] - 1s 107us/step - loss: 1.9290 - acc: 0.1760
Epoch 3/100
6012/6012 [==============================] - 1s 106us/step - loss: 1.9195 - acc: 0.1846
Epoch 4/100
6012/6012 [==============================] - 1s 105us/step - loss: 1.9166 - acc: 0.1813
Epoch 5/100
6012/6012 [==============================] - 1s 106us/step - loss: 1.9144 - acc: 0.1866
Epoch 6/100
6012/6012 [==============================] - 1s 107us/step - loss: 1.9126 - acc: 0.1939
Epoch 7/100
6012/6012 [==============================] - 1s 104us/step - loss: 1.9104 - acc: 0.1974
Epoch 8/100
6012/6012 [==============================] - 1s 108us/step - loss: 1.9091 - acc: 0.1896
Epoch 9/100
6012/6012 [==============================] - 1s 105us/step - loss: 1.9074 - acc: 0.1966
Epoch 10/100
6012/6012 [==============================] - 1s 107us/step - loss: 1.9059 - acc: 0.2008

6012/6012 [==============================] - 1s 106us/step - loss: 1.8688 - acc: 0.2400
Epoch 82/100
6012/6012 [==============================] - 1s 102us/step - loss: 1.8686 - acc: 0.2365
Epoch 83/100
6012/6012 [==============================] - 1s 102us/step - loss: 1.8685 - acc: 0.2364
Epoch 84/100
6012/6012 [==============================] - 1s 105us/step - loss: 1.8678 - acc: 0.2349
Epoch 85/100
6012/6012 [==============================] - 1s 104us/step - loss: 1.8687 - acc: 0.2387
Epoch 86/100
6012/6012 [==============================] - 1s 105us/step - loss: 1.8685 - acc: 0.2418
Epoch 87/100
6012/6012 [==============================] - 1s 106us/step - loss: 1.8681 - acc: 0.2374
Epoch 88/100
6012/6012 [==============================] - 1s 104us/step - loss: 1.8683 - acc: 0.2412
Epoch 89/100
6012/6012 [==============================] - 1s 103us/step - loss: 1.8683 - acc: 0.2377 0s - loss: 1.8645 - acc: 0
Epoch 90/100
6012/6012 [==============================] - 1s 104us/step - los

In [117]:
score = classifier.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1504/1504 [==============================] - 0s 49us/step
Test loss: 1.89124598148
Test accuracy: 0.217420212766


In [120]:
fake=['This umbrella sucks', 'I really like this! Great', 'Amazing, except for the handle', 'I wish this were larger', 'Works great on rainy days but it\'s too bulky', 'Never fails. EVER', 'adfhasdfjlasdklfjalsd;fkalskfjas;dkfjasl;']
fake_vector=vectorizer.transform(fake)
fake_matrix=fake_vector.todense()

In [121]:
classifier.predict(fake_matrix)

array([[ 0.08143406,  0.04536506,  0.07854054,  0.09341928,  0.2359508 ,
         0.40073335,  0.06455693],
       [ 0.13048436,  0.14304596,  0.14373942,  0.12209124,  0.17006916,
         0.17345317,  0.11711673],
       [ 0.14598057,  0.26013464,  0.15295328,  0.12297364,  0.06581549,
         0.05296037,  0.19918203],
       [ 0.16462198,  0.15502509,  0.10217686,  0.15080556,  0.12871711,
         0.15936249,  0.13929088],
       [ 0.07077152,  0.0507069 ,  0.16598909,  0.0820716 ,  0.33255103,
         0.24381307,  0.05409683],
       [ 0.17719789,  0.20145389,  0.14438334,  0.16100937,  0.09022938,
         0.05638948,  0.16933656],
       [ 0.14870912,  0.17725636,  0.0957453 ,  0.1254729 ,  0.12642367,
         0.19860101,  0.12779169]], dtype=float32)

In [133]:
fake_highest=['sadness', 'sadness and joy', 'disgust', 'anger (even split)' ,'joy and shame', 'disgust', 'joy']

0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame

In [203]:
class emotion_classify:
    def __init__(self):
        self.df = isear
        self.a = pd.Series('joy')
        self.b = pd.Series(['On days when I feel close to my partner and other friends.When I feel at peace with myself and also experience a close contact with people whom I regard greatly.'])
        self.new_df = pd.DataFrame({'Text': self.b, 'Emotion': self.a})
        self.stop = set(stopwords.words('english'))  ## stores all the stopwords in the lexicon
        self.exclude = set(string.punctuation)  ## stores all the punctuations
        self.lemma = WordNetLemmatizer()

        ## lets create a list of all negative-words
        self.negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']

        ## create a separate list to store texts and emotion
        self.em_list = []
        self.text_list = []

        ## create the training set
        self.train = []

         # stores the summarized text in a list
        self.sum_text_list = []

        # the e-score list stores the e-score for each document
        self.e_score_dict = {}

        # call the driver function
        self.main()

    '''
    A function for cleaning up all the documents
    # removes stop words
    # removes punctuations
    # uses lemmatizer 
    '''

    def clean(self, doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in self.stop if i not in self.negative])
        punc_free = "".join([ch for ch in stop_free if ch not in self.exclude])
        normalized = " ".join([self.lemma.lemmatize(word) for word in punc_free.split()])
        return normalized

    '''
    Function to iterate and clean up all texts
    '''

    def iterate_clean(self):
        for i in range(self.df.shape[0]):
            self.new_df.iloc[i]['Text'] = self.clean(self.new_df.iloc[i]['Text'])

    '''
    Function to iterate and populate text list
    '''

    def iterate_pop_text(self):
        for i in range(self.new_df.shape[0]):
            self.text_list.append(self.new_df.iloc[i]['Text'])

    '''
    Function to iterate and populate emotion list
    '''

    def iterate_pop_emotion(self):
        for i in range(self.new_df.shape[0]):
            self.em_list.append(self.new_df.iloc[i]['Emotion'])

    '''
    Function to create training set
    '''

    def create_train(self):
        for i in range(self.new_df.shape[0]):
            self.train.append([self.text_list[i], self.em_list[i]])

    '''
    Function to create model
    classify the query text
    and then summarize other texts
    classify them and return a dictionary containing the e-score for all documents
    '''

    def classify_text(self, text):

        cl = nltk.NaiveBayesClassifier(self.train)

        result = cl.classify(text)

        print(result)




    '''
    A function which is the driver for the entire class

    '''

    def main(self):

        self.iterate_clean()
        self.iterate_pop_emotion()
        self.iterate_pop_text()
        self.create_train()
        self.classify_text('I like pie')

In [204]:
nandi_obj=emotion_classify()

IndexError: single positional indexer is out-of-bounds

In [167]:
text_series=pd.Series(text)
full_df=pd.DataFrame()
full_df['text']=text_series
full_df['emotion']=isear[0]
full_df

,text,emotion
0,day feel close partner friend feel peace also ...,joy
1,every time imagine someone love could contact ...,fear
2,obviously unjustly treated possibility elucida...,anger
3,think short time live relate period life think...,sadness
4,gathering found involuntarily sitting next two...,disgust
5,realized directing feeling discontent partner ...,shame
6,feel guilty realize consider material thing im...,guilt
7,girlfriend taken exam went parent place,joy
8,when first time realized meaning death,fear
9,car overtaking another forced drive road,anger


In [185]:
featuresets=[]
def dict_docs(doc):
    return {'text': doc}
for index, doc in enumerate(text):
    featuresets.append((dict_docs(doc), isear[0].iloc[index]))

In [186]:
train_set, test_set= featuresets[:7000], featuresets[7000:]

In [187]:
train_set

[({'text': 'day feel close partner friend feel peace also experience close contact people regard greatly'},
  'joy'),
 ({'text': 'every time imagine someone love could contact serious illness even death'},
  'fear'),
 ({'text': 'obviously unjustly treated possibility elucidating this'},
  'anger'),
 ({'text': 'think short time live relate period life think use short time'},
  'sadness'),
 ({'text': 'gathering found involuntarily sitting next two people expressed opinion considered low discriminating'},
  'disgust'),
 ({'text': 'realized directing feeling discontent partner way trying put blame instead sorting feeliings'},
  'shame'),
 ({'text': 'feel guilty realize consider material thing important caring relative feel selfcentered'},
  'guilt'),
 ({'text': 'girlfriend taken exam went parent place'}, 'joy'),
 ({'text': 'when first time realized meaning death'}, 'fear'),
 ({'text': 'car overtaking another forced drive road'}, 'anger'),
 ({'text': 'recently thought hard work take study o

In [188]:
naivebayes=nltk.NaiveBayesClassifier.train(train_set)

In [192]:
print(nltk.classify.accuracy(naivebayes, test_set))

0.1511627906976744
